In [ ]:
!pip install -q -U transformers
!pip install sentencepiece

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... do

In [ ]:
from datasets import load_dataset

In [ ]:
# from accelerate import FullyShardedDataParallelPlugin, Accelerator
# from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

# fsdp_plugin = FullyShardedDataParallelPlugin(
#     state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
#     optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
# )

# accelerator = Accelerator(fsdp_plugin=fsdp_plugin)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-large"
#model_id = "openlm-research/open_llama_3b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# set tokenizer settings to save memory
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

## Metaphor Probabilities (zero-shot) for the pretrained model CODE

In [ ]:
import numpy as np
from scipy.special import softmax
import pdb
import pandas as pd
import math
from typing import List
import random
import argparse
import torch


def sent_scoring(model_tokenizer, text, cuda, score_type="loss", output_attentions=False, length_normalize=False):
    model = model_tokenizer[0]
    tokenizer = model_tokenizer[1]
    assert model is not None
    assert tokenizer is not None
    encoded_text = tokenizer.encode(text)
    input_ids = torch.tensor(encoded_text).unsqueeze(0)
    if cuda:
        input_ids = input_ids.to('cuda')
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids, output_attentions=output_attentions)
    loss, logits = outputs[:2]

    sentence_prob = loss.item()
    if score_type == "prob":
        if length_normalize:
            mult = 2
        else:
            mult = len(encoded_text)

        sentence_prob = math.exp(-1.0 * loss * (mult - 1))

    if output_attentions:
        attn = outputs["attentions"]
        return sentence_prob, attn, input_ids

    return sentence_prob

def confusion_matrix(P_forward_1, P_forward_2, P_backward_1, P_backward_2):
    correct_forward = len(np.where(np.array(P_forward_1) >= 0.5)[0]) + len(np.where(np.array(P_forward_2) >=0.5)[0])
    wrong_forward = len(P_forward_1) + len(P_forward_2) - correct_forward

    correct_backward = len(np.where(np.array(P_backward_1) >= 0.5)[0]) + len(np.where(np.array(P_backward_2) >=0.5)[0])
    wrong_backward = len(P_backward_1) + len(P_backward_2) - correct_backward

    print("correct forward", correct_forward, "wrong forward", wrong_forward, "correct backward", correct_backward, "wrong_backward", wrong_backward)

def evaluate_model(model, tokenizer, test_set, prompt_template = "{context}. {sentence}.", verbose=True, score_type="prob", use_cuda=False, return_acc=False) -> tuple:
    preds = []
    labels = []
    x_1 = []
    x_2 = []
    y_1 = []
    y_2 = []
    P_x_1 = []
    P_x_2 = []
    P_y_1 = []
    P_y_2 = []
    P_x_1_y_1 = []
    P_x_1_y_2 = []
    P_x_2_y_1 = []
    P_x_2_y_2 = []
    P_x_1_correct = []
    P_x_2_correct = []
    P_y_1_correct = []
    P_y_2_correct = []
    correct = 0

    for i, metaphor_data in enumerate(test_set):
        ctx, p1, p2 = metaphor_data["startphrase"], metaphor_data["ending1"], metaphor_data["ending2"]
        labels.append(int(metaphor_data["labels"]))

        try:
          sent1 =  prompt_template.format(context = ctx.rstrip("."), sentence = p1.rstrip("."))
          sent2 = prompt_template.format(context = ctx.rstrip("."), sentence = p2.rstrip("."))
        except:
          print(" ERROR: ", prompt_template)
          if return_acc:
              return None, None, None, None
          return None, None, None

        score1 = sent_scoring((model, tokenizer), sent1, use_cuda, score_type=score_type)
        score2 = sent_scoring((model, tokenizer), sent2, use_cuda, score_type=score_type)

        if score_type == "loss":
            pred = 0 if score1 < score2 else 1
        else:
            pred = 1 if score1 < score2 else 0

        pred_sent = sent1 if pred == 0 else sent2

        if i % 2 == 0:
            x_1.append(ctx)
            x_1_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_1.append(x_1_score)
            y_1.append(p1)
            y_2.append(p2)
            y1_score = sent_scoring((model, tokenizer), p1 + ".", use_cuda, score_type=score_type)
            y2_score = sent_scoring((model, tokenizer), p2 + ".", use_cuda, score_type=score_type)
            P_y_1.append(y1_score)
            P_y_2.append(y2_score)

            P_x_1_y_1.append(score1)
            P_x_1_y_2.append(score2)
            P_x_1_correct.append(score1/(score1 + score2))

        else:
            x_2.append(ctx)
            x_2_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_2.append(x_2_score)
            P_x_2_y_1.append(score1)
            P_x_2_y_2.append(score2)
            P_x_2_correct.append(score2/(score1 + score2))

            P_y_1_correct.append(P_x_1_y_1[-1]/(P_x_1_y_1[-1] + score1))
            P_y_2_correct.append(score2/(P_x_1_y_2[-1] + score2))

        if verbose:
            print(f"Q: {ctx}: 1. {p1} 2. {p2}")
            print(f"model says '{pred_sent}' is more likely")
            print("\n")
        if pred == metaphor_data["labels"]:
            correct += 1
        preds.append(pred)

    cols = {"x_1": x_1, "x_2": x_2, "y_1": y_1, "y_2": y_2, "P(x_1)": P_x_1, "P(x_2)": P_x_2, "P(y_1)": P_y_1, "P(y_2)": P_y_2,
        "P(x_1, y_1)": P_x_1_y_1, "P(x_1, y_2)": P_x_1_y_2, "P(x_2, y_1)": P_x_2_y_1, "P(x_2, y_2)": P_x_2_y_2,
        "P(y_1|x_1)": P_x_1_correct, "P(y_2|x_2)": P_x_2_correct, "P(x_1|y_1)": P_y_1_correct, "P(x_2|y_2)": P_y_2_correct}
    out_df = pd.DataFrame(cols)

    if return_acc:
        return correct/len(preds), out_df, preds, labels

    return out_df, preds, labels

def compute_stats(total_df: pd.DataFrame, all_preds: List, all_labels: List) -> None:
    print("overall accuracy: ")
    acc = len(np.where(np.array(all_preds) == np.array(all_labels))[0])/len(all_labels)
    print(acc)
    print("confusion matrix: ")
    confusion_matrix(list(total_df["P(y_1|x_1)"]), list(total_df["P(y_2|x_2)"]), list(total_df["P(x_1|y_1)"]), list(total_df["P(x_2|y_2)"]))

    x = {
        "P_forward_1": len(list(total_df["P(y_1|x_1)"])),
        "P_forward_2": len(list(total_df["P(y_2|x_2)"])),
        "P_backward_1":len(list(total_df["P(x_1|y_1)"])),
        "P_backward_2":len(list(total_df["P(x_2|y_2)"])),
        "accuracy": acc
    }
    return x

In [ ]:
dataset = load_dataset("nightingal3/fig-qa")
dataset['validation']

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['startphrase', 'ending1', 'ending2', 'labels', 'valid'],
    num_rows: 1094
})

# Run Prompts

In [ ]:
prompt_templates = [
"{context}. {sentence}.", # Base Score
"Translate from english:\n\n{context}.{sentence}", # Translation
"Translate from english to english without methaphors:\n\n{context}. Translation: {sentence}", # Translation
"Q: What is the english translation of {context}?\nA: {sentence}", # Translation question
"{context}\nThis is to say:\n{sentence}",    # Question Answering
"Question: {context}\nAnswer:{sentence}",    # Question Answering
"Can you interpret the following: {context}. What is the answer??\n{sentence}", # Question Answering
"Article:\n{context}Summary:\n{sentence}",    # Summarize
"Write a summary based on this article:\n\n{context}\n\n{sentence}", # Summarize
"Write highlights for this article:\n\n{context}\n\n{sentence}" #highlights summary?
"Find the right ending to this passage.\n\n{context}\n\nThis is to say \n\n{sentence}", # Passage writing / record

"Here is one sentence: {context}\nHere is another sentence: {sentence}\nDoes the term mean the same thing in both these sentences?\n", # wic - Sentence comparisons
"Here is one sentence: {context}\nDoes the term mean the same thing in both these sentences?\n{sentence}", # wic - Sentence comparisons
"Complete the following sentence.\n\n{context}\nThis is to say {sentence}. ", # wsc273 sentence completion
"How does this following sentence end?\n\n{context}\nThis is to say {sentence}.", # wsc273 sentence completion

"{context}\n\nDo \"{context}\" and \"{sentence}\" have the same meaning?\n\n", # wsc - part of sentence comparison

"Read the text and determine if the sentence is true:\n\n{context}\n\nSentence: {sentence}.", # Context guessing?  rte
"{context}\nCan we infer the following?\n{sentence}\n", # Context guessing?  rte
"{context}\nCan we infer the following?\nThis is to say\n\n{sentence}.", # Context guessing?  rte
"Concepts: {context}\n\nWrite a sentence that includes all these words.\n\n {sentence}." # common_gen - concept explaination
]

In [ ]:
subsubset_test_dataset = dataset['validation'].select(range(70))

result_middlephrase = {}
for prompt in prompt_templates:
  acc, out_df, preds, labels = evaluate_model(model, tokenizer, subsubset_test_dataset, prompt_template=prompt, return_acc = True)
  if preds == None:
      result_middlephrase[prompt] = {
          "x" : "Error in prompt",
          "accuracy":0
      }
      continue
  res = compute_stats(out_df, preds, labels)
  res["accuracy_2"] = acc
  result_middlephrase[prompt] = res


Streaming output truncated to the last 5000 lines.
Having that conversation was easy' is more likely


Q: That conversation had the ease of a Sunday morning.: 1. Having that conversation was difficult 2. Having that conversation was easy.
model says 'Write a summary based on this article:

That conversation had the ease of a Sunday morning

Having that conversation was easy' is more likely


Q: Their conversations were artillery bombardments.: 1. Their conversations were heated and antagonistic. 2. Their conversations were friendly.
model says 'Write a summary based on this article:

Their conversations were artillery bombardments

Their conversations were heated and antagonistic' is more likely


Q: Their conversations were a hug with words.: 1. Their conversations were heated and antagonistic. 2. Their conversations were friendly.
model says 'Write a summary based on this article:

Their conversations were a hug with words

Their conversations were friendly' is more likely


Q: The s

In [ ]:
# BASE with empty prompt
result_middlephrase

{'{context}. {sentence}.': {'P_forward_1': 35,
  'P_forward_2': 35,
  'P_backward_1': 35,
  'P_backward_2': 35,
  'accuracy': 0.6285714285714286,
  'accuracy_2': 0.6285714285714286},
 'Translate from english:\n\n{context}.{sentence}': {'P_forward_1': 35,
  'P_forward_2': 35,
  'P_backward_1': 35,
  'P_backward_2': 35,
  'accuracy': 0.5428571428571428,
  'accuracy_2': 0.5428571428571428},
 'Translate from english to english without methaphors:\n\n{context}. Translation: {sentence}': {'P_forward_1': 35,
  'P_forward_2': 35,
  'P_backward_1': 35,
  'P_backward_2': 35,
  'accuracy': 0.45714285714285713,
  'accuracy_2': 0.45714285714285713},
 'Q: What is the english translation of {context}?\nA: {sentence}': {'P_forward_1': 35,
  'P_forward_2': 35,
  'P_backward_1': 35,
  'P_backward_2': 35,
  'accuracy': 0.5571428571428572,
  'accuracy_2': 0.5571428571428572},
 '{context}\nThis is to say:\n{sentence}': {'P_forward_1': 35,
  'P_forward_2': 35,
  'P_backward_1': 35,
  'P_backward_2': 35,
  '

In [ ]:
max_accuracy = max(result_middlephrase, key=lambda x: result_middlephrase[x]['accuracy'])
max_accuracy_value = result_middlephrase[max_accuracy]['accuracy']

# Find all prompts with the maximum accuracy value
prompts_with_max_accuracy = [index for index, entry in result_middlephrase.items() if entry['accuracy'] == max_accuracy_value]

print(f"The maximum accuracy is {max_accuracy_value} at index/indices {', '.join(map(str, prompts_with_max_accuracy))}")

The maximum accuracy is 0.7142857142857143 at index/indices Can you interpret the following: {context}. What is the answer??
{sentence}, {context}
Can we infer the following?
This is to say

{sentence}.


In [ ]:
# Log results
result_middlephrase

{'{context}. {sentence}.': {'P_forward_1': 35,
  'P_forward_2': 35,
  'P_backward_1': 35,
  'P_backward_2': 35,
  'accuracy': 0.6285714285714286,
  'accuracy_2': 0.6285714285714286},
 'Translate from english:\n\n{context}.{sentence}': {'P_forward_1': 35,
  'P_forward_2': 35,
  'P_backward_1': 35,
  'P_backward_2': 35,
  'accuracy': 0.5428571428571428,
  'accuracy_2': 0.5428571428571428},
 'Translate from english to english without methaphors:\n\n{context}. Translation: {sentence}': {'P_forward_1': 35,
  'P_forward_2': 35,
  'P_backward_1': 35,
  'P_backward_2': 35,
  'accuracy': 0.45714285714285713,
  'accuracy_2': 0.45714285714285713},
 'Q: What is the english translation of {context}?\nA: {sentence}': {'P_forward_1': 35,
  'P_forward_2': 35,
  'P_backward_1': 35,
  'P_backward_2': 35,
  'accuracy': 0.5571428571428572,
  'accuracy_2': 0.5571428571428572},
 '{context}\nThis is to say:\n{sentence}': {'P_forward_1': 35,
  'P_forward_2': 35,
  'P_backward_1': 35,
  'P_backward_2': 35,
  '

In [ ]:
#  Print top Results
result_list = [[k,v["accuracy"]] for k,v in result_middlephrase.items()]
result_df = pd.DataFrame(result_list, columns=["prompt", "accuracy"])
sorted_df = result_df.sort_values(by='accuracy', ascending=False)
sorted_df


,prompt,accuracy
17,{context}\nCan we infer the following?\nThis i...,0.714286
6,Can you interpret the following: {context}. Wh...,0.714286
4,{context}\nThis is to say:\n{sentence},0.685714
14,"{context}\n\nDo ""{context}"" and ""{sentence}"" h...",0.657143
10,Here is one sentence: {context}\nHere is anoth...,0.642857
16,{context}\nCan we infer the following?\n{sente...,0.642857
0,{context}. {sentence}.,0.628571
5,Question: {context}\nAnswer:{sentence},0.628571
15,Read the text and determine if the sentence is...,0.600000
3,Q: What is the english translation of {context...,0.557143


# Run fine tuned prompts

In [ ]:
best_prompts = [
 # wsc273 sentence completion
"Complete the following sentence.\n\n{context}\nThis is to say {sentence}. ",
"How does this following sentence end?\n\n{context}\nThis is to say {sentence}.", # wsc273 sentence completion


"{context}\nDoes this next sentence follow, given the preceding text?\nThis is to say \n\n {sentence}",
"{context}\nCan we infer the following?\nThis is to say\n\n{sentence}.", # Context guessing?  rte
"Think carefully before answering: can we draw the following hypothesis from the premise\nContext:\n\n{context}\n\nHypothesis: This is to say\n{sentence}",
"{context}\n\nCan we draw the following conclusion?\nThis is to say \n\n{sentence}",

"Can you interpret the following: {context}. What is the answer??\n{sentence}",# Question Answering

 # Base Scores
"{context}. {sentence}.",
"{context}. That is to say, {sentence}.",
]


if len(best_prompts) != 9:
  raise Exception ("Check , at end again")

In [ ]:
subsubset_test_dataset = dataset['validation'].select(range(500))

result_best_prompts = {}
for prompt in best_prompts:
  acc, out_df, preds, labels = evaluate_model(model, tokenizer, subsubset_test_dataset, prompt_template=prompt, return_acc = True)
  if preds == None:
      result_best_prompts[prompt] = {
          "x" : "Error in prompt",
          "accuracy":0
      }
      continue
  res = compute_stats(out_df, preds, labels)
  res["accuracy_2"] = acc
  result_best_prompts[prompt] = res


Streaming output truncated to the last 5000 lines.
The floor was slippery' is more likely


Q: The floor was as slippery as sandpaper: 1. The floor was slippery. 2. The floor wasn't slippery at all.
model says 'Can you interpret the following: The floor was as slippery as sandpaper. What is the answer??
The floor was slippery' is more likely


Q: The gem was as fake as a bad toupee: 1. The gem was obviously fake. 2. The gem was obviously real.
model says 'Can you interpret the following: The gem was as fake as a bad toupee. What is the answer??
The gem was obviously fake' is more likely


Q: The gem was as fake as the air we breathe: 1. The gem was obviously fake. 2. The gem was obviously real.
model says 'Can you interpret the following: The gem was as fake as the air we breathe. What is the answer??
The gem was obviously fake' is more likely


Q: The boys are as disciplined as the military: 1. The boys are highly disciplined. 2. The boys lack any discipline.
model says 'Can you inter

In [ ]:
#  Print top Results
best_prompts_result_list = [[k,v["accuracy"]] for k,v in result_best_prompts.items()]
best_prompts_result_df = pd.DataFrame(best_prompts_result_list, columns=["prompt", "accuracy"])
best_prompts_sorted_df = best_prompts_result_df.sort_values(by='accuracy', ascending=False)
best_prompts_sorted_df

,prompt,accuracy
5,{context}\n\nCan we draw the following conclus...,0.698
2,"{context}\nDoes this next sentence follow, giv...",0.680
3,{context}\nCan we infer the following?\nThis i...,0.658
4,Think carefully before answering: can we draw ...,0.636
7,{context}. {sentence}.,0.630
6,Can you interpret the following: {context}. Wh...,0.622
8,"{context}. That is to say, {sentence}.",0.618
0,Complete the following sentence.\n\n{context}\...,0.530
1,How does this following sentence end?\n\n{cont...,0.520


In [ ]:
#base:
print(best_prompts[-2], result_best_prompts[best_prompts[-2]])
print(best_prompts[-1],result_best_prompts[best_prompts[-1]])

{context}. {sentence}. {'P_forward_1': 250, 'P_forward_2': 250, 'P_backward_1': 250, 'P_backward_2': 250, 'accuracy': 0.63, 'accuracy_2': 0.63}
{context}. That is to say, {sentence}. {'P_forward_1': 250, 'P_forward_2': 250, 'P_backward_1': 250, 'P_backward_2': 250, 'accuracy': 0.618, 'accuracy_2': 0.618}


In [ ]:
acc, out_df, preds, labels = evaluate_model(model, tokenizer, subsubset_test_dataset, prompt_template="{context}. That is to say {sentence}.", return_acc = True)
acc

Q: The girl had the flightiness of a sparrow: 1. The girl was very fickle. 2. The girl was very stable.
model says 'The girl had the flightiness of a sparrow. That is to say The girl was very fickle.' is more likely


Q: The girl had the flightiness of a rock: 1. The girl was very fickle. 2. The girl was very stable.
model says 'The girl had the flightiness of a rock. That is to say The girl was very fickle.' is more likely


Q: It was as peaceful as a church.: 1. It was very peaceful. 2. It was full of conflict and danger, not peace.
model says 'It was as peaceful as a church. That is to say It was very peaceful.' is more likely


Q: It was as peaceful as a battlefield.: 1. It was very peaceful. 2. It was full of conflict and danger, not peace.
model says 'It was as peaceful as a battlefield. That is to say It was full of conflict and danger, not peace.' is more likely


Q: The leaves were as green as grass: 1. The leaves were very green 2. The leaves were brown and not green at all.


0.64

In [ ]:
acc, out_df, preds, labels = evaluate_model(model, tokenizer, subsubset_test_dataset, prompt_template="{context} that is to say {sentence}.", return_acc = True)
acc

Q: The girl had the flightiness of a sparrow: 1. The girl was very fickle. 2. The girl was very stable.
model says 'The girl had the flightiness of a sparrow that is to say The girl was very fickle.' is more likely


Q: The girl had the flightiness of a rock: 1. The girl was very fickle. 2. The girl was very stable.
model says 'The girl had the flightiness of a rock that is to say The girl was very fickle.' is more likely


Q: It was as peaceful as a church.: 1. It was very peaceful. 2. It was full of conflict and danger, not peace.
model says 'It was as peaceful as a church that is to say It was very peaceful.' is more likely


Q: It was as peaceful as a battlefield.: 1. It was very peaceful. 2. It was full of conflict and danger, not peace.
model says 'It was as peaceful as a battlefield that is to say It was full of conflict and danger, not peace.' is more likely


Q: The leaves were as green as grass: 1. The leaves were very green 2. The leaves were brown and not green at all.
mode

0.572